In [43]:
import pandas as pd 

In [44]:
from sqlalchemy import create_engine

In [45]:
def lire_donnees_postgresql(table_name, engine):
    """
    Lit les données depuis une table PostgreSQL et remplace les caractères spéciaux dans les colonnes.

    :param table_name: Nom de la table à lire
    :param engine: Objet SQLAlchemy engine pour se connecter à la base de données
    :return: DataFrame Pandas avec les données de la table et les caractères spéciaux retirés
    """
    # Lire les données depuis la base de données
    df = pd.read_sql(f"SELECT * FROM {table_name}", engine)

    # Retirer les caractères spéciaux (exemple : \xa0 qui représente un espace insécable)
    df.replace(r"\xa0", "", regex=True, inplace=True)

    return df

In [46]:
# Informations de connexion à PostgreSQL
username = 'user-ayasaidi'       # Remplacez par votre nom d'utilisateur
password = '0cryo6kzligjfi3aw3p6'  # Remplacez par votre mot de passe
hostname = 'postgresql-795529.user-ayasaidi'      # Ou l'adresse IP de votre serveur PostgreSQL
database = 'defaultdb'      # Nom de votre base de données

# Créer l'engine SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{hostname}/{database}')

# Appeler la fonction pour lire les données depuis la table 'usagers'
df_vehicules = lire_donnees_postgresql('vehicules', engine)

# Afficher les premières lignes du DataFrame
print(df_vehicules.head())

        Num_Acc id_vehicule num_veh  senc  catv  obs  obsm  choc  manv  motor  \
0  202200000001      813952     A01     1     2    0     2     1     9      1   
1  202200000001      813953     B01     1     7    0     2     2     1      1   
2  202200000002      813950     B01     2     7    0     2     8    15      1   
3  202200000002      813951     A01     2    10    0     2     1     1      1   
4  202200000003      813948     A01     2     7    0     2     1     2      1   

   occutc  
0     NaN  
1     NaN  
2     NaN  
3     NaN  
4     NaN  


La méthode pd.read_sql, bien qu'efficace pour lire des données directement depuis une base de données dans un DataFrame Pandas, n'est généralement pas recommandée pour être utilisée dans une API pour plusieurs raisons :

Performance :

pd.read_sql charge toute la table ou la requête SQL en mémoire (dans un DataFrame). Cela peut être inefficace si la table contient un grand nombre d'enregistrements. Dans un environnement API, cela peut conduire à des problèmes de performance et à des temps de réponse longs, surtout si les requêtes sont fréquentes ou les tables volumineuses.
Les API doivent souvent répondre en temps réel à de nombreuses requêtes. Si chaque requête lit l'intégralité d'une table en mémoire, cela peut engorger le système.
Gestion de la mémoire :

Pandas est conçu pour fonctionner principalement en mémoire (RAM). Si la base de données contient une grande quantité de données, cela peut entraîner une consommation excessive de mémoire, augmentant le risque de ralentissements ou de pannes du serveur.
Une API doit être conçue pour répondre de manière optimale avec un minimum de consommation de ressources.
Scalabilité :

Les API doivent être conçues pour être scalables. Utiliser pd.read_sql dans une API signifie que chaque appel API pourrait provoquer une lourde opération de lecture de base de données, rendant difficile la mise à l'échelle de l'application (par exemple, lorsqu'il y a de nombreux utilisateurs concurrents).
Les ORM (Object Relational Mapping) comme SQLAlchemy, qui permettent de charger des données de manière plus sélective et d'utiliser des requêtes plus optimisées, sont préférés dans le cadre d'une API.
Simplicité des requêtes :

Lors de la création d'une API, il est souvent préférable de manipuler les données directement avec SQL ou un ORM comme SQLAlchemy pour écrire des requêtes optimisées, qui permettent de récupérer uniquement les données nécessaires et non toute la table ou des blocs de données non filtrés.

In [47]:
print(df_vehicules.columns)

Index(['Num_Acc', 'id_vehicule', 'num_veh', 'senc', 'catv', 'obs', 'obsm',
       'choc', 'manv', 'motor', 'occutc'],
      dtype='object')


In [48]:
def get_vehicle_by_id(df, id_vehicule):
    """
    Renvoie la première ligne correspondant à l'id_vehicule sous forme de dictionnaire.
    :param df: DataFrame contenant les données
    :param id_vehicule: L'identifiant du véhicule à chercher
    :return: Dictionnaire de la première ligne correspondant à l'id_vehicule
    """
    # Filtrer le DataFrame pour trouver les lignes correspondant à id_vehicule
    print(df.columns)
    result = df[df['id_vehicule'] == id_vehicule]
    
    # Si une ligne est trouvée, la renvoyer sous forme de dictionnaire
    if not result.empty:
        return result.iloc[0].to_dict()
    
    # Sinon, renvoyer un dictionnaire vide
    return {}    

In [49]:
df_vehicules

,Num_Acc,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor,occutc
0,202200000001,813952,A01,1,2,0,2,1,9,1,NaN
1,202200000001,813953,B01,1,7,0,2,2,1,1,NaN
2,202200000002,813950,B01,2,7,0,2,8,15,1,NaN
3,202200000002,813951,A01,2,10,0,2,1,1,1,NaN
4,202200000003,813948,A01,2,7,0,2,1,2,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...
94488,202200055300,715633,A01,2,7,2,0,1,1,1,NaN
94489,202200055301,715631,A01,2,7,0,0,8,19,1,NaN
94490,202200055301,715632,B01,2,7,0,2,1,1,1,NaN
94491,202200055302,715629,A01,1,33,0,2,1,1,1,NaN


In [50]:
# Tester avec un id_vehicule existant
result_1 = get_vehicle_by_id(df_vehicules, "813953")
print(f"Résultat pour id_vehicule 813953 : {result_1}")

# Tester avec un id_vehicule non existant
result_2 = get_vehicle_by_id(df_vehicules, "8000000")
print(f"Résultat pour id_vehicule 8000000 : {result_2}")

Index(['Num_Acc', 'id_vehicule', 'num_veh', 'senc', 'catv', 'obs', 'obsm',
       'choc', 'manv', 'motor', 'occutc'],
      dtype='object')
Résultat pour id_vehicule 813953 : {'Num_Acc': 202200000001, 'id_vehicule': '813953', 'num_veh': 'B01', 'senc': 1, 'catv': 7, 'obs': 0, 'obsm': 2, 'choc': 2, 'manv': 1, 'motor': 1, 'occutc': nan}
Index(['Num_Acc', 'id_vehicule', 'num_veh', 'senc', 'catv', 'obs', 'obsm',
       'choc', 'manv', 'motor', 'occutc'],
      dtype='object')
Résultat pour id_vehicule 8000000 : {}


In [51]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Welcome to FastAPI!"}

@app.get("/items/{item_id}")
def read_item(item_id: int, q: str = None):
    return {"item_id": item_id, "query": q}